In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import FastMarkerCluster
import osmnx as ox
import matplotlib.pyplot as plt
%matplotlib inline

### Analisi delle ciclabili

In [ ]:
gdf_ciclabili = gpd.read_file('MappaFinaleLivelliSicurezza.geojson', crs={'init': 'epsg:4326'}, geometry='geometry')

In [ ]:
#Ho notato che ci sono dei pessi di ciclabili che hanno lo stesso id di OSM anche se sono separati tra di loro, quindi vado 
#a modificare gli id in modo da renderli univoci 

#l_ids = list(gdf_ciclabili['id'])

l_ids = [x for x in range(gdf_ciclabili.shape[0])]

#for i in range(len(l_ids)):
#    l_ids[i] = l_ids[i] + "-"+str(i)
    
gdf_ciclabili.insert(2, 'id_num', l_ids)
gdf_ciclabili.head()

In [ ]:
#Visto che in Neo4j Spatial non c'è un modo rapido e veloce per decretare se sue poligoni si toccano, uso la funzione 'touches' 
#per determinare quali bicycle lanes si toccano tra di loro e aggiungo questa informazione in una colonna


list_touched_lanes = []
for _, r in gdf_ciclabili.iterrows():
    list_touched_lanes.append(list(gdf_ciclabili[gdf_ciclabili['geometry'].touches(r['geometry'])]['id_num']))
    
    
gdf_ciclabili['touched_lanes'] = list_touched_lanes

#Adesso nel GeoDataFrame gdf_ciclabili ho tutte le informazioni che mi servono per creare il grafo delle ciclabili
gdf_ciclabili.to_csv('ciclabili.csv')

### Analisi degli attraversamenti (crossing)

In [ ]:
gdf_crossing_nodes =  gpd.read_file('crossing_nodes.geojson', crs={'init': 'epsg:4326'}, geometry='geometry')
gdf_crossing_nodes = gdf_crossing_nodes[gdf_crossing_nodes.geometry.type != "Polygon"]
gdf_crossing_nodes.head(10)

In [ ]:
#Inserisco l'id numerico anche ai crossing nodes
l_ids = [x for x in range(gdf_crossing_nodes.shape[0])]

#for i in range(len(l_ids)):
#    l_ids[i] = l_ids[i] + "-"+str(i)
    
gdf_crossing_nodes.insert(2, 'id_num', l_ids)
gdf_crossing_nodes.head()

In [ ]:
gdf_crossing_nodes.to_file("crossing_nodes.geojson")

In [ ]:
gdf_crossing_ways =  gpd.read_file('crossing_ways.geojson', crs={'init': 'epsg:4326'}, geometry='geometry')
gdf_crossing_ways = gdf_crossing_ways[gdf_crossing_ways.geometry.type != "Polygon"]
gdf_crossing_ways.head(10)

In [ ]:
#Ho notato che ci sono dei pessi di ciclabili che hanno lo stesso id di OSM anche se sono separati tra di loro, quindi vado 
#a modificare gli id in modo da renderli univoci 

#l_ids = list(gdf_ciclabili['id'])

l_ids = [x for x in range(gdf_crossing_ways.shape[0])]

#for i in range(len(l_ids)):
#    l_ids[i] = l_ids[i] + "-"+str(i)
    
gdf_crossing_ways.insert(2, 'id_num', l_ids)
gdf_crossing_ways.head()

In [ ]:
#Creaiamo un GeoDataFrame provvisorio relativo alle ciclabili che ci servirà successivamente per fare dei calcoli sulle distanze
#tra le ciclabili e gli attraversamenti

gdf_ciclabili_prova = gpd.GeoDataFrame(gdf_ciclabili)
gdf_ciclabili_prova.to_crs(epsg=3310, inplace=True)

In [ ]:
#Gli attraversamenti del tipo ways o intersecano o sono molto vicini alle bicycle lanes, quindi per determinare se è possibile
#attraversare devo trovare gli attraversamenti più vicini alla ciclabile di riferimento, mentre le intersezioni le posso 
#calcolare direttamente su Neo4j grazie alla funzione spatial.intersects
gdf_crossing_ways_prova = gpd.GeoDataFrame(gdf_crossing_ways)
gdf_crossing_ways_prova.to_crs(epsg=3310, inplace=True)

s = gdf_ciclabili_prova['geometry']
list_closest_lanes = []

for index, r in gdf_crossing_ways_prova.iterrows(): 
    print(index)
    polygon = gdf_crossing_ways_prova[gdf_crossing_ways_prova['id'] == r.id]['geometry'].iloc[0]
    l = list(s.distance(polygon))
    #print(l)
    l1 = []
    for i in range(len(l)):
    #    l1 = []
        if l[i] <= 2:
            l1.append(gdf_ciclabili_prova.iloc[i].id_num)
    list_closest_lanes.append(l1)


In [ ]:
gdf_crossing_ways['closest_lanes'] = list_closest_lanes
gdf_crossing_ways.to_crs(epsg=4326, inplace=True)